In [27]:
def offer_parser(soup, position):
    import re
    from bs4 import BeautifulSoup

    
    results = []
        # wyciągnięcie wszystkich ofert pracy na stronie
    offers_lst = soup.find_all('a', class_="posting-list-item")
        # print(len(offers_lst)) # testowo sprawdzniee, czy jest 20 (dla pierwszej strony)

        # parsujemy dane z oferty
    for offer in offers_lst:
        offer_name = offer.find("h3", class_='posting-title__position').text.strip()
        offer_e = offer.find('span', class_='posting-title__company').text.strip()
        offer_emp = offer_e[2:]

        offer_salary = offer.find('span', class_='text-truncate badgy salary btn btn-outline-secondary btn-sm'
        ).text
        offer_salary_data = offer_salary.split('-')
        offer_salary_data_min = offer_salary_data[0].replace(' ', '')
        offer_salary_data_max = offer_salary_data[1][:-4].replace(' ', '')
        offer_salary_data_currency = offer_salary_data[1][-4:]

        offer_technology = offer.find('a', class_='btn btn-outline-secondary btn-sm text-truncate')
        if offer_technology is None:
            offer_technology = 'N/A'
        else:
            offer_technology = offer_technology.text.strip()

            # TODO
        offer_loc = offer.find(class_='posting-info__location')
        location_data = offer_loc.text.split(',')

            # sprawdzenie jakiego miasta dtyczy oferta
        if len(location_data) > 1:
            offer_loc_country = location_data[1][1:4]
                #offer_loc_country_nam = re.search("\w+", offer_loc_country)
            offer_loc_city = location_data[0]
        else:
            offer_loc_country = 'N/A'
            offer_loc_city = 'Zdalna'

        offer_data = {
            'location': {
                'city': offer_loc_city,
                'coutry': offer_loc_country
            },
            'salary': {
                'low': offer_salary_data_min,
                'high': offer_salary_data_max,
                'currency': offer_salary_data_currency
            },
            'name': offer_name,
            'company': offer_emp,
            'technology': offer_technology,
            'job': position[0]
        }
        results.append(offer_data)
    return(results)

In [28]:
def parser():
    import re
    from datetime import datetime
    from glob import glob
    import pandas as pd
    from bs4 import BeautifulSoup
    
    
    html_lst = glob(r'..\data\raw\*.html')
    
    n = 0
    results = []
    for html_f in html_lst:
        with open(html_f, mode="r", encoding="UTF-8") as f:
            html = f.read()

        soup = BeautifulSoup(html, 'html.parser')
        #print(soup)
        patt = r"\w+\s[a-z]+"
        position = re.findall(patt, html_f, flags=re.IGNORECASE)
        results += offer_parser(soup, position)
        n+=1

        #results = offer_parser(html_lst)
        d = datetime.now().strftime('%Y_%m_%d')
        df_raw = pd.json_normalize(results, sep='_')
        df_raw.to_csv(    
        f'../data/interim/job_offers_{d}_{n}.csv',
        sep=';',
        encoding='UTF-8',
        index=False
        )

In [29]:
parser()